# 2. [Clair](https://github.com/coreos/clair) と [Klar](https://github.com/optiopay/klar) で Docker イメージを脆弱性スキャン

実際に脆弱性スキャンしてみましょう！

## 2.1. ソフトウェアとそのアーキテクチャについて

### Clair

### Klar

### AWS 公式にある構成事例

## 2.2. Docker で CLI をラップする

## 2.3. 一般 Docker イメージのスキャン

ワンオフ実行とも言われる、[コマンドを Docker でラップしたもの](https://github.com/supinf/dockerized-tools/blob/master/golang/klar/versions/2.4/Dockerfile) を使い、スキャンを実行してみます。

In [ ]:
source ~/config/.env

docker run --rm \
      -e CLAIR_ADDR="http://${API_HOST}:6060" \
      supinf/klar:2.4 envoyproxy/envoy-alpine:v1.10.0 | jq .

成功時応答）
```text
{
  "LayerCount": 6,
  "Vulnerabilities": {}
}
```

脆弱性をもつ Docker イメージで、緊急度を **Medium** にして試してみましょう。

In [ ]:
docker run --rm \
      -e CLAIR_ADDR="http://${API_HOST}:6060" -e CLAIR_OUTPUT=Medium \
      supinf/klar:2.4 envoyproxy/envoy:v1.10.0 | jq .

成功時応答例）
```text
{
  "LayerCount": 9,
  "Vulnerabilities": {
    "Medium": [
      {
        "Name": "CVE-2009-5155",
        "NamespaceName": "ubuntu:16.04",
        "Description": "In the GNU C Library (aka glibc or libc6) before 2.28, .."
        ..
      },
      {
        "Name": "CVE-2019-9893",
        "NamespaceName": "ubuntu:16.04",
        "Description": "libseccomp before 2.4.0 ..",
        ..
      }
    ]
  }
}
```

## 2.4. ECR においた Docker イメージのスキャン

ECR へ任意の Docker イメージを push し、それに対してスキャンをかけてみます。

In [ ]:
image_name=
docker run --rm \
      -e AWS_ACCESS_KEY_ID -e AWS_SECRET_ACCESS_KEY -e AWS_DEFAULT_REGION \
      -e CLAIR_ADDR="http://${API_HOST}:6060" -e CLAIR_OUTPUT=Low \
      supinf/klar:2.4 ${image_name} | jq .

[次へ: 03-trivy](03-trivy.ipynb)